In [146]:
#Jai Ganesh Deva

## <u><i> Convolutional Neural Network for Sarcasm Detection </u></i>

<br>

- We have used the CNN architecture proposed Yoon  (https://arxiv.org/abs/1408.5882) Kim to classify sarcastic sentences.<br>
- We have experimented with Word2Vec



In [147]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize  import word_tokenize
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn import pipeline,metrics, grid_search
from gensim import models, corpora
from gensim.models import ldamodel
import os
import matplotlib.pyplot as plt
%matplotlib inline
import collections
import io
import keras.backend as K
import csv
import numpy as np
import pandas as pd
import dill as pickle
import gensim
from keras.optimizers import Adam
from gensim.models import word2vec,doc2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn import metrics

In [148]:
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import  Dense, Input,Conv1D,Conv2D,Convolution1D,Convolution2D ,LSTM, Embedding, Dropout, Activation,Concatenate
from keras.layers import Bidirectional, Masking,MaxPooling1D,MaxPooling2D,MaxPool2D,GlobalMaxPool1D,Bidirectional,Merge,Flatten,Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


import gc

In [149]:
os.chdir("E:\\Datasets\\Final Project\\Politics Final\\test_balanced")

In [150]:
X_train = pd.read_csv("train_ndf.csv")

In [151]:
X_test = pd.read_csv("test_ndf.csv")

In [152]:
train_Y = np.array(X_train["sarcastic or not"])

In [153]:
test_Y = np.array(X_test["sarcastic or not"])

In [154]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("wordvec_SARC", binary=True)


In [155]:
def f1_score(y_true, y_pred):
    # Recall metric. Only computes a batch-wise average of recall,
    # a metric for multi-label classification of how many relevant items are selected.
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    # Precision metric. Only computes a batch-wise average of precision,
    # a metric for multi-label classification of how many selected items are relevant.
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [156]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
   
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
   
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [157]:
def comment_to_vec(tokens, model, features):
   
    
    feature_vec = np.zeros((features), dtype="float32")
    word_count = 0
    
   
    ind2word = set(model.wv.index2word)
    
    for word in tokens:
        if word in ind2word: 
            word_count += 1
            feature_vec += model[word]

    feature_vec /= word_count
    return feature_vec

In [158]:
def gen_comment_vecs(comments, model, features ):
    current_index = 0
    
    comment_feature_vec = np.zeros((len(comments), features), dtype="float32")

    for comment in comments:
        if current_index%1000 == 0.:
            print ("Vectorizing review %d of %d" % (current_index, len(comments)))
   
        comment_feature_vec[current_index] = comment_to_vec(comment, model, features)
        current_index += 1
       
    return comment_feature_vec
    

In [159]:
train_embedding = gen_comment_vecs(X_train["text_tokens_stop"],word2vec,100)

Vectorizing review 0 of 13668


C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


Vectorizing review 1000 of 13668
Vectorizing review 2000 of 13668
Vectorizing review 3000 of 13668
Vectorizing review 4000 of 13668
Vectorizing review 5000 of 13668
Vectorizing review 6000 of 13668
Vectorizing review 7000 of 13668
Vectorizing review 8000 of 13668
Vectorizing review 9000 of 13668
Vectorizing review 10000 of 13668
Vectorizing review 11000 of 13668
Vectorizing review 12000 of 13668
Vectorizing review 13000 of 13668


In [160]:
test_embedding = gen_comment_vecs(X_test["text_tokens_stop"],word2vec,100)   

Vectorizing review 0 of 2840
Vectorizing review 1000 of 2840


C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


Vectorizing review 2000 of 2840


In [161]:
tokenizer = Tokenizer(num_words=30000, lower=False, char_level=False)
tokenizer.fit_on_texts(X_train["lower_wp"].tolist())
training_sequences = tokenizer.texts_to_sequences(X_train["lower_wp"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=200)

train_embedding_weights = np.zeros((len(train_word_index)+1, 100))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(100)
print(train_embedding_weights.shape)

Found 13200 unique tokens.
(13201, 100)


In [162]:
trial_1 = tokenizer.texts_to_matrix(X_train["lower_wp"].tolist(),mode="tfidf")

In [163]:
trail_2 = tokenizer.texts_to_matrix(X_test["lower_wp"].tolist(),mode = "tfidf")

In [164]:
trial_1_sequences = pad_sequences(trial_1,maxlen=200)

In [165]:
trial_2_sequences = pad_sequences(trail_2,maxlen = 200)

In [166]:
test_cnn_data

array([[   0,    0,    0, ...,   40,    4, 4987],
       [   0,    0,    0, ...,   18,   86,  784],
       [   0,    0,    0, ...,  655,    5,  685],
       ...,
       [   0,    0,    0, ...,   90,  352, 5612],
       [   0,    0,    0, ...,  221,   14,   53],
       [   0,    0,    0, ..., 2761, 1125, 2315]])

In [167]:
train_cnn_data

array([[    0,     0,     0, ...,     0,   278,   787],
       [    0,     0,     0, ...,    28,    57,  2180],
       [    0,     0,     0, ...,   125,  3505,   875],
       ...,
       [    0,     0,     0, ...,     3,     1,   286],
       [    0,     0,     0, ...,    12,     1, 13199],
       [    0,     0,     0, ..., 13200,   233,    16]])

In [168]:
test_sequences = tokenizer.texts_to_sequences(X_test["lower_wp"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=200)

In [178]:
def conv_sent_sarc(embeddings, max_sequence_length, num_words, embedding_dim, labels_index, trainable=False, extra_conv=True):
    
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)

   
    convs = []
    filter_sizes = [3,4,5]

    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=128, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=3)(l_conv)
        convs.append(l_pool)

    l_merge = Merge(mode='concat', concat_axis=1)(convs)

    
    conv = Conv1D(filters=128, kernel_size=3, activation='relu')(embedded_sequences)
    pool = MaxPooling1D(pool_size=3)(conv)

    if extra_conv==True:
        x = Dropout(0.5)(l_merge)  
    else:
       
        x = Dropout(0.5)(pool)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
   
    preds = Dense(labels_index, activation='sigmoid')(x)

    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [179]:
x_train = train_cnn_data
y_tr = train_Y

In [180]:
model = conv_sent_sarc(train_embedding_weights, 200, len(train_word_index)+1, 100, 
                1, False)

C:\Users\vamsi\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 200, 100)     1320100     input_13[0][0]                   
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 198, 128)     38528       embedding_14[0][0]               
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 197, 128)     51328       embedding_14[0][0]               
__________________________________________________________________________________________________
conv1d_14 

In [143]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)


In [144]:
hist_shuffled = model.fit(train_cnn_data, train_Y, epochs=10, callbacks=[early_stopping], validation_split=0.1, shuffle=True, batch_size=256)


#12301/12301 [==============================] - 35s 3ms/step - loss: 1.0874 - acc: 0.6438 - val_loss: 1.1804 - val_acc: 0.6079

Train on 12301 samples, validate on 1367 samples
Epoch 1/10
12301/12301 [==============================] - 33s 3ms/step - loss: 1.1497 - acc: 0.5226 - val_loss: 1.3003 - val_acc: 0.5465
Epoch 2/10
12301/12301 [==============================] - 33s 3ms/step - loss: 1.1456 - acc: 0.5426 - val_loss: 1.2858 - val_acc: 0.5633
Epoch 3/10
12301/12301 [==============================] - 39s 3ms/step - loss: 1.1322 - acc: 0.5497 - val_loss: 1.3028 - val_acc: 0.5523
Epoch 4/10
12301/12301 [==============================] - 37s 3ms/step - loss: 1.1493 - acc: 0.5648 - val_loss: 1.3891 - val_acc: 0.5655
Epoch 5/10
12301/12301 [==============================] - 39s 3ms/step - loss: 1.1407 - acc: 0.5756 - val_loss: 1.3726 - val_acc: 0.5801
Epoch 6/10
12301/12301 [==============================] - 38s 3ms/step - loss: 1.1459 - acc: 0.5856 - val_loss: 1.3603 - val_acc: 0.5596
Epoch 00006: early stopping


In [59]:
result  = model.evaluate(test_cnn_data,test_Y,batch_size=32, verbose=1)

2840/2840 [==============================] - 3s 959us/step


In [60]:
result

[1.2158717628935694, 0.5827464850855545]

In [ ]:
#GLove 

In [181]:
EMBEDDING_FILE = "E:\Datasets\Final Project\Politics Final\glove.twitter.27B.25d.txt"

In [185]:
#Will Take some Time to load
embeddings_index = dict()

f = open(EMBEDDING_FILE,encoding = "utf8")
for line in f:
    
    values = line.split()
   
    word = values[0]
    
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs #50 dimensions
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

KeyboardInterrupt: 

In [183]:
gc.collect()

all_embs = np.stack(list(embeddings_index.values()))
emb_mean,emb_std = all_embs.mean(), all_embs.std()

ValueError: need at least one array to stack

In [ ]:
nb_words = len(tokenizer.word_index)
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
gc.collect()

In [ ]:
embeddedCount = 0
for word, i in tokenizer.word_index.items():
    i-=1
    
    embedding_vector = embeddings_index.get(word)
    .
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector
        embeddedCount+=1
print('total embedded:',embeddedCount,'common words')

del(embeddings_index)
gc.collect()


return embedding_matrix

In [ ]:
tokenizer = Tokenizer(num_words=30000, lower=False, char_level=False)
tokenizer.fit_on_texts(X_train["lower_wp"].tolist())
training_sequences = tokenizer.texts_to_sequences(X_train["lower_wp"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

train_cnn_data = pad_sequences(training_sequences, maxlen=200)

train_embedding_weights = np.zeros((len(train_word_index)+1, 100))


In [ ]:
model = conv_sent_sarc([embedding_matrix], 200, len(train_word_index)+1, 25, 
                1, False)

In [ ]:
result = model.evaluate(test_cnn_data,test_Y,batch_size=32, verbose=1)

In [ ]:
from keras.utils.vis_utils import plot_model

In [175]:

plot_model(model, to_file='CNN.png', show_shapes=True, show_layer_names=True)